In [1]:
cd ..

/home/admin-hieunn/important/VNUIS-Chatbot


/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import textwrap
import time
import sys
import locale
import os
from dotenv import load_dotenv
import json

load_dotenv()
# Load models and data once
embedding_path = os.getenv("PROJECTCB1_EMBEDDING_MODEL")
embedding_model = SentenceTransformer(
    model_name_or_path=embedding_path,
    model_kwargs={"torch_dtype": "bfloat16"},
    device="cuda",
    trust_remote_code=True,
)
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

rerank_tokenizer = AutoTokenizer.from_pretrained("Model/bge-reranker-v2-m3")
rerank_model = AutoModelForSequenceClassification.from_pretrained(
    "Model/bge-reranker-v2-m3", torch_dtype=torch.bfloat16, device_map="cuda:0"
)
rerank_model.eval()


# Load embeddings and data once
def load_embeddings(embeddings_path, device):
    try:
        text_chunks_and_embedding_df = pd.read_csv(embeddings_path)

        # Convert the embedding column from JSON strings to lists of floats
        text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df[
            "embedding"
        ].apply(json.loads)

        # Convert embeddings to PyTorch tensors
        embeddings = torch.tensor(
            np.array(text_chunks_and_embedding_df["embedding"].tolist()),
            dtype=torch.bfloat16,
        ).to(device)

        pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
    except Exception as e:
        raise RuntimeError(f"Failed to load embeddings: {e}")

    return embeddings, pages_and_chunks


embeddings, pages_and_chunks = load_embeddings(
    embeddings_path="Data/DB/ChunksAndEmbedding.csv", device="cuda"
)


# Function to retrieve relevant resources
def retrieve_relevant_resources(
    query: str,
    number_result_embedding: int = 20,
    number_result_reranking: int = 5,
    threshold: int = -4,
):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    # Get top scores with a threshold
    # scores, indices = torch.topk(input=cosine_scores, k=n_resources_to_return)
    scores, indices = torch.topk(input=dot_scores, k=number_result_embedding)
    print(scores)

    context_items = [pages_and_chunks[i] for i in indices]
    results = [item["Relevant docs"] for item in context_items]

    pairs = [[query, result] for result in results]

    with torch.no_grad():
        inputs = rerank_tokenizer(
            pairs, padding=True, truncation=True, return_tensors="pt", max_length=1024
        )
        inputs = {
            key: value.to("cuda") for key, value in inputs.items()
        }  # Move all inputs to the same device as the model

        # Compute scores
        rerank_scores = rerank_model(**inputs, return_dict=True).logits.view(
            -1,
        )

        top_scores, top_indices = torch.topk(rerank_scores, k=number_result_reranking)
        # Help me add script to only take the score > -3
        filtered_indices = top_indices[top_scores > threshold]
        rerank_result = [results[i] for i in filtered_indices]

    return results, scores, top_scores, rerank_result
    # return rerank_result

/home/admin-hieunn/important/VNUIS-Chatbot/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [13]:
query = "Giới thiệu về cô Trần Thị Oanh"
results, scores, top_scores, rerank_result = retrieve_relevant_resources(
    query=query, number_result_embedding=50, number_result_reranking=3, threshold = -3.5)

tensor([0.6055, 0.5977, 0.5898, 0.5859, 0.5625, 0.5586, 0.5586, 0.5586, 0.5547,
        0.5547, 0.5547, 0.5547, 0.5508, 0.5508, 0.5508, 0.5469, 0.5469, 0.5469,
        0.5430, 0.5430, 0.5430, 0.5430, 0.5352, 0.5352, 0.5312, 0.5312, 0.5273,
        0.5273, 0.5234, 0.5195, 0.5195, 0.5117, 0.5117, 0.5078, 0.5078, 0.5078,
        0.5078, 0.5078, 0.5039, 0.5039, 0.5000, 0.4980, 0.4980, 0.4961, 0.4961,
        0.4961, 0.4863, 0.4785, 0.4785, 0.4688], device='cuda:0',
       dtype=torch.bfloat16)


In [14]:
results

['Tôi muốn nghiên cứu chủ đề "Khai phá dữ liệu (Data Mining)", tôi nên chọn giảng viên hướng dẫn là ai?\nHướng nghiên cứu của Cô Trần Thị Oanh là về lĩnh vực Computer Science (Khoa học máy tính)\n1. Khai phá dữ liệu (Data Mining)\n2. Ứng dụng máy học trong kinh doanh (Applications of machine learning in business)\n3. Xử lý ngôn ngữ tự nhiên (Natural Language Processing)',
 'Tôi muốn nghiên cứu chủ đề "Xử lý ngôn ngữ tự nhiên (Natural Language Processing)", tôi nên chọn giảng viên hướng dẫn là ai?\nHướng nghiên cứu của Cô Trần Thị Oanh là về lĩnh vực Computer Science (Khoa học máy tính)\n1. Khai phá dữ liệu (Data Mining)\n2. Ứng dụng máy học trong kinh doanh (Applications of machine learning in business)\n3. Xử lý ngôn ngữ tự nhiên (Natural Language Processing)',
 'Tôi muốn nghiên cứu chủ đề "Ứng dụng máy học trong kinh doanh (Applications of machine learning in business)", tôi nên chọn giảng viên hướng dẫn là ai?\nHướng nghiên cứu của Cô Trần Thị Oanh là về lĩnh vực Computer Science (K

In [15]:
scores

tensor([0.6055, 0.5977, 0.5898, 0.5859, 0.5625, 0.5586, 0.5586, 0.5586, 0.5547,
        0.5547, 0.5547, 0.5547, 0.5508, 0.5508, 0.5508, 0.5469, 0.5469, 0.5469,
        0.5430, 0.5430, 0.5430, 0.5430, 0.5352, 0.5352, 0.5312, 0.5312, 0.5273,
        0.5273, 0.5234, 0.5195, 0.5195, 0.5117, 0.5117, 0.5078, 0.5078, 0.5078,
        0.5078, 0.5078, 0.5039, 0.5039, 0.5000, 0.4980, 0.4980, 0.4961, 0.4961,
        0.4961, 0.4863, 0.4785, 0.4785, 0.4688], device='cuda:0',
       dtype=torch.bfloat16)

In [16]:
top_scores

tensor([1.7422, 1.2031, 0.6094], device='cuda:0', dtype=torch.bfloat16)

In [17]:
rerank_result

['Thông tin liên hệ cơ bản của PGS. TS.Trần Thị Oanh là gì?\nPGS. TS.Trần Thị Oanh:\n\nChức vụ: Phó trưởng Khoa\nHọc hàm, học vị: Phó Giáo sư  Tiến sĩ\nNgoại ngữ: Tiếng Anh\nEmail: tranthioanh@vnu.edu.vn\nĐịa chỉ:Khoa Các khoa học ứng dụng, P408, nhà C, Làng sinh viên HACINCO, 79 Ngụy Như Kon Tum, Nhân Chính, Thanh Xuân',
 'Chức vụ của PGS. TS.Trần Thị Oanh là gì?\nPGS. TS.Trần Thị Oanh:\n\nChức vụ: Phó trưởng Khoa\nHọc hàm, học vị: Phó Giáo sư  Tiến sĩ\nNgoại ngữ: Tiếng Anh\nEmail: tranthioanh@vnu.edu.vn\nĐịa chỉ:Khoa Các khoa học ứng dụng, P408, nhà C, Làng sinh viên HACINCO, 79 Ngụy Như Kon Tum, Nhân Chính, Thanh Xuân',
 'Bằng cấp của PGS. TS. Trần Thị Oanh là gì?\nPGS. TS.Trần Thị Oanh:\n\nCử nhân: Công nghệ Thông tin, Đại học Công nghệ – ĐHQG Hà Nội\nThạc sĩ: Hệ thống thông tin, Đại học Công nghệ – ĐHQG Hà Nội\nTiến sĩ: Khoa học Máy tính, Viện khoa học và công nghệ tiên tiến Nhật Bản']